In [1]:
import pandas as pd
import numpy as np
import torch
import scripts
from functools import lru_cache
import torchmetrics
from torch import nn
import optuna

/home/karolyi/miniconda/envs/project_thesis/lib/python3.13/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Data loading

First we load the data. The basic idea is to create dictionaries with features associated to the drugs and cell-lines. In principle, the splits and the data shouldn't be changed

In [2]:
@lru_cache(maxsize = None)
def get_data(n_fold = 0, fp_radius = 2):
    smile_dict = pd.read_csv("data/smiles.csv", index_col=0)
    fp = scripts.FingerprintFeaturizer(R = fp_radius)
    drug_dict = fp(smile_dict.iloc[:, 1], smile_dict.iloc[:, 0])
    driver_genes = pd.read_csv("data/driver_genes.csv").loc[:, "symbol"].dropna()
    rnaseq = pd.read_csv("data/rnaseq_normcount.csv", index_col=0)
    driver_columns = rnaseq.columns.isin(driver_genes)
    filtered_rna = rnaseq.loc[:, driver_columns]
    tensor_exp = torch.Tensor(filtered_rna.to_numpy())
    cell_dict = {cell: tensor_exp[i] for i, cell in enumerate(filtered_rna.index.to_numpy())}
    data = pd.read_csv("data/GDSC12.csv", index_col=0)
    # default, remove data where lines or drugs are missing:
    data = data.query("SANGER_MODEL_ID in @cell_dict.keys() & DRUG_ID in @drug_dict.keys()")
    unique_cell_lines = data.loc[:, "SANGER_MODEL_ID"].unique()
    np.random.seed(420) # for comparibility, don't change it!
    np.random.shuffle(unique_cell_lines)
    folds = np.array_split(unique_cell_lines, 10)
    test_lines = folds[0]
    train_idxs = list(range(10))
    train_idxs.remove(n_fold)
    np.random.seed(420)
    validation_idx = np.random.choice(train_idxs)
    train_idxs.remove(validation_idx)
    train_lines = np.concatenate([folds[idx] for idx in train_idxs])
    validation_lines = folds[validation_idx]
    test_lines = folds[n_fold]
    train_data = data.query("SANGER_MODEL_ID in @train_lines")
    validation_data = data.query("SANGER_MODEL_ID in @validation_lines")
    test_data = data.query("SANGER_MODEL_ID in @test_lines")
    return (scripts.OmicsDataset_drugwise(cell_dict, drug_dict, train_data),
    scripts.OmicsDataset_drugwise(cell_dict, drug_dict, validation_data),
    scripts.OmicsDataset_drugwise(cell_dict, drug_dict, test_data))

In [3]:
@lru_cache(maxsize = None)
def get_data_original(n_fold = 0, fp_radius = 2):
    smile_dict = pd.read_csv("data/smiles.csv", index_col=0)
    fp = scripts.FingerprintFeaturizer(R = fp_radius)
    drug_dict = fp(smile_dict.iloc[:, 1], smile_dict.iloc[:, 0])
    driver_genes = pd.read_csv("data/driver_genes.csv").loc[:, "symbol"].dropna()
    rnaseq = pd.read_csv("data/rnaseq_normcount.csv", index_col=0)
    driver_columns = rnaseq.columns.isin(driver_genes)
    filtered_rna = rnaseq.loc[:, driver_columns]
    tensor_exp = torch.Tensor(filtered_rna.to_numpy())
    cell_dict = {cell: tensor_exp[i] for i, cell in enumerate(filtered_rna.index.to_numpy())}
    data = pd.read_csv("data/GDSC12.csv", index_col=0)
    # default, remove data where lines or drugs are missing:
    data = data.query("SANGER_MODEL_ID in @cell_dict.keys() & DRUG_ID in @drug_dict.keys()")
    unique_cell_lines = data.loc[:, "SANGER_MODEL_ID"].unique()
    np.random.seed(420) # for comparibility, don't change it!
    np.random.shuffle(unique_cell_lines)
    folds = np.array_split(unique_cell_lines, 10)
    test_lines = folds[0]
    train_idxs = list(range(10))
    train_idxs.remove(n_fold)
    np.random.seed(420)
    validation_idx = np.random.choice(train_idxs)
    train_idxs.remove(validation_idx)
    train_lines = np.concatenate([folds[idx] for idx in train_idxs])
    validation_lines = folds[validation_idx]
    test_lines = folds[n_fold]
    train_data = data.query("SANGER_MODEL_ID in @train_lines")
    validation_data = data.query("SANGER_MODEL_ID in @validation_lines")
    test_data = data.query("SANGER_MODEL_ID in @test_lines")
    return (scripts.OmicsDataset(cell_dict, drug_dict, train_data),
    scripts.OmicsDataset(cell_dict, drug_dict, validation_data),
    scripts.OmicsDataset(cell_dict, drug_dict, test_data))

# Configuration

we declare the configuration, this is going to be model-specific and we get the datasets

In [4]:
config = {"features" : {"fp_radius":2},
          "optimizer": {"batch_size": 2,
                        "clip_norm":19,
                        "learning_rate":0.0004592646200179472,
                        "stopping_patience":15},
          "model":{"embed_dim":485,
                 "hidden_dim":696,
                 "dropout":0.48541242824674574,
                 "n_layers": 4,
                 "norm": "batchnorm",},
         "env": {"fold": 0,
                 "device":"cpu",
                 "max_epochs": 100,
                 "search_hyperparameters":True}}

In [5]:
train_dataset, validation_dataset, test_dataset = get_data(n_fold = config["env"]["fold"],
                                                           fp_radius = config["features"]["fp_radius"])

train_dataset_og, validation_dataset_og, test_dataset_og = get_data_original(n_fold = config["env"]["fold"],
                                                           fp_radius = config["features"]["fp_radius"])

[15:55:13] DEPRECATION WARNING: please use MorganGenerator
[15:55:13] DEPRECATION WARNING: please use MorganGenerator
[15:55:13] DEPRECATION WARNING: please use MorganGenerator
[15:55:13] DEPRECATION WARNING: please use MorganGenerator
[15:55:13] DEPRECATION WARNING: please use MorganGenerator
[15:55:13] DEPRECATION WARNING: please use MorganGenerator
[15:55:13] DEPRECATION WARNING: please use MorganGenerator
[15:55:13] DEPRECATION WARNING: please use MorganGenerator
[15:55:13] DEPRECATION WARNING: please use MorganGenerator
[15:55:13] DEPRECATION WARNING: please use MorganGenerator
[15:55:13] DEPRECATION WARNING: please use MorganGenerator
[15:55:13] DEPRECATION WARNING: please use MorganGenerator
[15:55:13] DEPRECATION WARNING: please use MorganGenerator
[15:55:13] DEPRECATION WARNING: please use MorganGenerator
[15:55:13] DEPRECATION WARNING: please use MorganGenerator
[15:55:13] DEPRECATION WARNING: please use MorganGenerator
[15:55:13] DEPRECATION WARNING: please use MorganGenerat

In [6]:
cell_features, drug_features, target, drug_index, cell_indices, labels = train_dataset[4]

ins = train_dataset[7]
print(ins[5].shape)
print(ins[4].shape)

print("Drug Features Shape:", drug_features.shape)
print("Cell Features Shape:", cell_features.shape)  
print("Labels Shape:", labels.shape)
print("Target Shape:", target.shape)               
print("Drug Index Shape:", drug_index.shape)      
print("Cell Indices Shape:", cell_indices.shape) 



c_f, d_f, target, cellid, drugid = train_dataset_og[0]

print("Drug Features Shape:", d_f.shape)
print("Cell Features Shape:", c_f.shape)
print("Target Shape:", target.shape)
print("Cell ID Shape:", cellid.shape)
print("Drug ID Shape:", drugid.shape)


torch.Size([1000])
torch.Size([1000])
Drug Features Shape: torch.Size([1000, 2048])
Cell Features Shape: torch.Size([1000, 777])
Labels Shape: torch.Size([1000])
Target Shape: torch.Size([1000])
Drug Index Shape: torch.Size([1000])
Cell Indices Shape: torch.Size([1000])
........
Drug Features Shape: torch.Size([2048])
Cell Features Shape: torch.Size([777])
Target Shape: torch.Size([1])
Cell ID Shape: torch.Size([1])
Drug ID Shape: torch.Size([1])


# Hyperparameter optimization

we wrap the function for training the model in a function that can be used by optuna

In [ ]:
def train_model_optuna(trial, config):
    def pruning_callback(epoch, train_r):
        trial.report(train_r, step = epoch)
        if np.isnan(train_r):
            raise optuna.TrialPruned()
        if trial.should_prune():
            raise optuna.TrialPruned()
    config["model"] = {"embed_dim": trial.suggest_int("embed_dim", 64, 512),
                    "hidden_dim": trial.suggest_int("hidden_dim", 64, 2048),
                    "n_layers": trial.suggest_int("n_layers", 1, 6),
                    "norm": trial.suggest_categorical("norm", ["batchnorm", "layernorm", None]),
                    "dropout": trial.suggest_float("dropout", 0.0, 0.5)}
    config["optimizer"] = { "learning_rate": trial.suggest_float("learning_rate", 1e-6, 1e-1, log=True),
                            "clip_norm": trial.suggest_int("clip_norm", 0.1, 20),
                            "batch_size": trial.suggest_int("batch_size", 5, 10),
                            "stopping_patience":10}
    try:
        R, model = scripts.train_model(config,
                                       train_dataset,
                                       validation_dataset,
                                       use_momentum=True,
                                       callback_epoch = pruning_callback)
        return R
    except Exception as e:
        print(e)
        return 0

In [10]:


if config["env"]["search_hyperparameters"]:
    study_name = f"baseline_model"
    storage_name = "sqlite:///studies/{}.db".format(study_name)
    study = optuna.create_study(study_name=study_name,
                                storage=storage_name,
                                direction='maximize',
                                load_if_exists=True,
                                pruner=optuna.pruners.MedianPruner(n_startup_trials=30,
                                                               n_warmup_steps=5,
                                                               interval_steps=5))
    objective = lambda x: train_model_optuna(x, config)
    study.optimize(objective, n_trials=40)
    best_config = study.best_params
    print(best_config)
    config["model"]["embed_dim"] = best_config["embed_dim"]
    config["model"]["hidden_dim"] = best_config["hidden_dim"]
    config["model"]["n_layers"] = best_config["n_layers"]
    config["model"]["norm"] = best_config["norm"]
    config["model"]["dropout"] = best_config["dropout"]
    config["optimizer"]["learning_rate"] = best_config["learning_rate"]
    config["optimizer"]["clip_norm"] = best_config["clip_norm"]
    config["optimizer"]["batch_size"] = best_config["batch_size"]



[I 2024-12-08 17:47:09,204] Using an existing study with name 'baseline_model' instead of creating a new one.


torch.Size([2048])


[I 2024-12-08 17:47:13,105] Trial 320 finished with value: 0.0 and parameters: {'embed_dim': 174, 'hidden_dim': 2025, 'n_layers': 4, 'norm': 'layernorm', 'dropout': 0.4144201574718056, 'learning_rate': 3.0603068308590013e-06, 'clip_norm': 0, 'batch_size': 10}. Best is trial 0 with value: 0.0.


torch.Size([1000, 777])
mat1 and mat2 shapes cannot be multiplied (1000x777000 and 2048000x174)
torch.Size([2048])
torch.Size([1000, 777])
mat1 and mat2 shapes cannot be multiplied (1000x777000 and 2048000x501)


[I 2024-12-08 17:47:23,158] Trial 321 finished with value: 0.0 and parameters: {'embed_dim': 501, 'hidden_dim': 1888, 'n_layers': 4, 'norm': None, 'dropout': 0.23372843845945868, 'learning_rate': 0.024582435559445222, 'clip_norm': 6, 'batch_size': 10}. Best is trial 0 with value: 0.0.


torch.Size([2048])


[I 2024-12-08 17:47:24,406] Trial 322 finished with value: 0.0 and parameters: {'embed_dim': 66, 'hidden_dim': 1518, 'n_layers': 2, 'norm': 'batchnorm', 'dropout': 0.3450734071994024, 'learning_rate': 1.207352136309009e-06, 'clip_norm': 20, 'batch_size': 10}. Best is trial 0 with value: 0.0.


Expected more than 1 value per channel when training, got input size torch.Size([1, 1518])
torch.Size([2048])


[W 2024-12-08 17:47:29,608] Trial 323 failed with parameters: {'embed_dim': 390, 'hidden_dim': 1384, 'n_layers': 5, 'norm': 'layernorm', 'dropout': 0.47504515800016034, 'learning_rate': 1.3191095666105825e-05, 'clip_norm': 19, 'batch_size': 10} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/home/karolyi/miniconda/envs/project_thesis/lib/python3.13/site-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_2732759/989827914.py", line 11, in <lambda>
    objective = lambda x: train_model_optuna(x, config)
                          ~~~~~~~~~~~~~~~~~~^^^^^^^^^^^
  File "/tmp/ipykernel_2732759/2571496089.py", line 18, in train_model_optuna
    R, model = scripts.train_model(config,
               ~~~~~~~~~~~~~~~~~~~^^^^^^^^
                                   train_dataset,
                                   ^^^^^^^^^^^^^^
                                   validation_dataset,
  

KeyboardInterrupt: 

# Model training and evaluation

After we have a set of optimal hyperparameters we train our model. The train model function could be changed, but:
- test_dataset cannot be used until we call the final evaluation step
- the evaluation step cannot be modified, it must take the model produced by your pipeline, a dataloader that provides the correct data for your model, and the final metrics have to be printed

In [11]:
import scripts

%reload_ext autoreload

%autoreload 2


_, model = scripts.train_model(config, torch.utils.data.ConcatDataset([train_dataset, validation_dataset]), None, use_momentum=False)
device = torch.device(config["env"]["device"])
metrics = torchmetrics.MetricTracker(torchmetrics.MetricCollection(
    {"R_cellwise_residuals":scripts.GroupwiseMetric(metric=torchmetrics.functional.pearson_corrcoef,
                          grouping="drugs",
                          average="macro",
                          residualize=True),
    "R_cellwise":scripts.GroupwiseMetric(metric=torchmetrics.functional.pearson_corrcoef,
                          grouping="cell_lines",
                          average="macro",
                          residualize=False),
    "MSE":torchmetrics.MeanSquaredError()}))
metrics.to(device)
test_dataloader = torch.utils.data.DataLoader(test_dataset,
                                       batch_size=config["optimizer"]["batch_size"],
                                       drop_last=False,
                                      shuffle=False)


/home/karolyi/PROJECT/scripts/drug_wise_base_model.py:219: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
/home/karolyi/PROJECT/scripts/drug_wise_base_model.py:220: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  drug_ids1 = torch.tensor(x[4][0])
/home/karolyi/PROJECT/scripts/drug_wise_base_model.py:221: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  cell_ids1 = torch.tensor(x[5][0])
/home/karolyi/PROJECT/scripts/drug_wise_base_model.py:223: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() 

torch.Size([1, 1000, 2])
torch.Size([1, 1000, 2])
torch.Size([1, 1000, 2])


RuntimeError: running_mean should contain 1000 elements not 696

In [15]:
final_metrics = scripts.evaluate_step(model, test_dataloader, metrics, device)
print(final_metrics)

{'MSE': 1.870725154876709, 'R_cellwise': 0.886543869972229, 'R_cellwise_residuals': 0.29409846663475037}
